In [1]:
# import libraries
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns 
import html
from bs4 import BeautifulSoup
import requests
import os
from selenium import webdriver
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import warnings; warnings.simplefilter('ignore')
from VE_scraper_functions import *
from chromedriver_py import binary_path # this will get you the path variable
from selenium.webdriver.common.by import By
import time
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Daniel.Lang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# import dataset
path = "park_scrape_content_dataset0705.xlsx"
park_scrape_dataset2 = pd.read_excel(path)
park_scrape_dataset2.head

<bound method NDFrame.head of        Unnamed: 0  index                                       website page                                            content                                   website location  park
0               0      0  https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...  NPS.govPark HomePlan Your VisitDirections & Tr...  ABLI
1               1      1  https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...  NPS.govPark HomePlan Your VisitDirections & Tr...  ABLI
2               2      2  https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...  NPS.govPark HomePlan Your VisitDirections & Tr...  ABLI
3               3      3  https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...  NPS.govPark HomePlan Your VisitDirections & Tr...  ABLI
4               4      4  https://home.nps.gov/abl

In [3]:
## convert all text to lowercase to avoid case sensitivity issues
park_scrape_dataset2['content'] = park_scrape_dataset2['content'].str.lower()
park_scrape_dataset2.head

<bound method NDFrame.head of        Unnamed: 0  index                                       website page                                            content                                   website location  park
0               0      0  https://home.nps.gov/abli/planyourvisit/direct...  directions - abraham lincoln birthplace nation...  NPS.govPark HomePlan Your VisitDirections & Tr...  ABLI
1               1      1  https://home.nps.gov/abli/planyourvisit/direct...  directions - abraham lincoln birthplace nation...  NPS.govPark HomePlan Your VisitDirections & Tr...  ABLI
2               2      2  https://home.nps.gov/abli/planyourvisit/direct...  directions - abraham lincoln birthplace nation...  NPS.govPark HomePlan Your VisitDirections & Tr...  ABLI
3               3      3  https://home.nps.gov/abli/planyourvisit/direct...  directions - abraham lincoln birthplace nation...  NPS.govPark HomePlan Your VisitDirections & Tr...  ABLI
4               4      4  https://home.nps.gov/abl

In [4]:
RH_words =  [
       " uber " ," lyft ", "ridehailing", "ridesharing", " cab ", " taxi ","ride-hailing" , "ride-sharing" , "ride hailing" ,
    "ride sharing", "ride share", "rideshare"]

In [ ]:
def count_RH_words(text):
    word_counts = {word: text.lower().count(word) for word in RH_words}
    return word_counts
word_counts_df = park_scrape_dataset2['content'].apply(count_RH_words).apply(pd.Series)
word_counts_df['total'] = word_counts_df.sum(axis=1)
word_counts_df.head()

In [ ]:
park_scrape_dataset2 = pd.concat([park_scrape_dataset2, word_counts_df], axis=1)
park_scrape_dataset2 = park_scrape_dataset2.drop("content", axis="columns")
park_scrape_dataset2.head

In [ ]:
park_scrape_dataset2.to_csv("RHtest.csv")

In [ ]:
path = "clusterlookup.csv"
clusters = pd.read_csv(path)
clusters.head

In [ ]:
park_word_counts = park_scrape_dataset2.groupby('park')[RH_words].sum()
park_word_counts['total'] = park_word_counts.sum(axis=1)
park_word_counts.head

In [ ]:
clustersubset = clusters[['Park Alpha', "FinalCluster", "Label"]]
clustersubset.rename(columns={'Park Alpha':"park"}, inplace = True)
clusterparks = pd.merge(park_word_counts, clustersubset, on="park", how="left")
clusterparks.head

In [ ]:
clusterparks.to_csv("RHbypark.csv")

In [ ]:
total_parks_counts = clusterparks.groupby('Label').size().reset_index(name='Total_Parks')
total_parks_counts['Total_Parks'] = total_parks_counts['Total_Parks'].astype(int) 
filtered_df = clusterparks[clusterparks['total'] > 1]
ev_parks_counts = filtered_df.groupby('Label').size().reset_index(name='RH_Parks')
label_counts = pd.merge(total_parks_counts, ev_parks_counts, on='Label', how='left')
label_counts['RH_Parks'].fillna(0, inplace=True)
label_counts['Percent_RH'] = round((label_counts['RH_Parks'] / label_counts['Total_Parks']) * 100, 1)
label_counts['RH_Parks'] = label_counts['RH_Parks'].astype(int)
label_counts.head()

In [ ]:
total_row = label_counts.sum(numeric_only=True)
total_row['Label'] = 'Total'
total_counts = label_counts.append(total_row, ignore_index=True)
total_counts['RH_Parks'] = total_counts['RH_Parks'].astype(int)
total_counts['Total_Parks'] = total_counts['Total_Parks'].astype(int)
total_counts.loc[total_counts['Label'] == 'Total', 'Percent_RH'] = round((total_counts.loc[total_counts['Label'] == 'Total', 'RH_Parks'] / total_counts.loc[total_counts['Label'] == 'Total', 'Total_Parks']) * 100, 1)
total_counts.tail()

In [ ]:
total_counts.to_csv("RHbycluster.csv")

In [ ]:
plt.figure(figsize=(10, 6)) 
plt.bar(label_counts['Label'], label_counts['Percent_RH'], color='#C56C39')
plt.xlabel('Cluster')
plt.ylabel('Percentage of Ridehailing Parks')
plt.title('Percentage of Ridehailing Parks in Each Cluster')
plt.xticks(rotation=45, ha='right') 
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6)) 
bars = plt.bar(label_counts['Label'], label_counts['Percent_RH'], color='#C56C39')

for i, bar in enumerate(bars):
    ev_parks = label_counts['RH_Parks'].iloc[i]
    total_parks = label_counts['Total_Parks'].iloc[i]
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.25,
             f"{ev_parks}/{total_parks}",
             ha='center', va='bottom')
    
##avg_percent_ev = label_counts['Percent_EV'].mean()
##plt.axhline(y=avg_percent_ev, color='red', linestyle='--')

plt.xlabel('Cluster')
plt.ylabel('Percentage of Parks with Ridehailing Information')
plt.title('Percentage of Parks with Ridehailing Information by Cluster')
plt.xticks(rotation=45, ha='right') 
plt.ylim(0, 35)
plt.tight_layout()
plt.savefig("RHGraph.png")
plt.show()